In [5]:
import requests
import pandas as pd
import json
import time
import os
import datetime
from pandas.io.json import json_normalize
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from pytz import timezone

In [39]:
class ccMaster:
    def __init__(self,url,keyspace,api_key='',coinSymbol='',fsym='',tsym='',cr=0):
        self.keyspace = keyspace
        self.createSessionCassandra(self.keyspace)
        
        if api_key:
            url += '&api_key={}'.format(api_key)
            self.api_key = api_key
        if coinSymbol:
            self.coinSymbol = coinSymbol
            self.coinID = self.getCoinID(self.coinSymbol)
            url += '&coinId={}'.format(self.coinID)
        if fsym:
            if cr == 0:
                self.coinSymbol = fsym
                self.coinID = self.getCoinID(self.coinSymbol)
            url += '&fsym={}'.format(fsym)
            self.fysm = fsym
        if tsym:
            if cr == 1:
                self.coinSymbol = tsym
                self.coinID = self.getCoinID(self.coinSymbol)
            url += '&tsym={}'.format(tsym)
            self.tysm = tsym
        
        self.url = url + '&limit=1513'
        print(self.url )
    
    def createSessionCassandra(self,key_space):
        self.cluster = Cluster(['127.0.0.1'],load_balancing_policy=None)
        self.session = self.cluster.connect()
        self.session.set_keyspace(key_space)
        
    def getCoinID(self,coinSymbol,tablename="all_coins"):
        query = "SELECT id FROM " + tablename + " WHERE symbol='" + coinSymbol + "' ALLOW FILTERING"
        future = self.session.execute(query)
        return future.one()[0]
    
    def getJSON(self):
        req=requests.get(self.url)
        json_obj = req.json()
        return json_obj
    
    def jsonToPandas(self,json_obj):
        df = json_normalize(json_obj)
        df['id'] = self.coinID
        df['symbol'] = str(self.coinSymbol)
        return df
        
    def checkTable(self,table_name,create_table_stmt):        
        self.session.execute("""CREATE TABLE IF NOT EXISTS """ + table_name + create_table_stmt)
        
    def convertToPst(self,inp_time):
        return datetime.datetime.fromtimestamp(int(inp_time)).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S')
        
    def pdToCassandra(self,table_name,create_table_stmt,insert_stmt):
        self.checkTable(table_name,create_table_stmt)
        query = """INSERT INTO """+ table_name + insert_stmt
        prepared = self.session.prepare(query)
        return prepared,self.session

In [6]:
top10 = ['BTC','XRP','ETH','LTC','BCH','EOS','ADA','XLM','IOT','NEO']
api_key = '23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41'
key_space = 'crypton'

In [386]:
# DB realted variables FOR social_status_day_hist
create_table_social_status_day_hist = """(
analysis_page_views INT,charts_page_views INT,code_repo_closed_issues INT,code_repo_closed_pull_issues INT,
code_repo_forks INT,code_repo_open_issues INT,code_repo_open_pull_issues INT,code_repo_stars INT,
code_repo_subscribers INT,comments INT,fb_likes INT,fb_talking_about INT,followers INT,forum_page_views INT,
influence_page_views INT,markets_page_views INT,overview_page_views INT,points INT,posts INT,
reddit_active_users INT,reddit_comments_per_day FLOAT,reddit_comments_per_hour FLOAT,reddit_posts_per_day FLOAT,
reddit_posts_per_hour FLOAT,reddit_subscribers INT,time TEXT,total_page_views INT,trades_page_views INT,
twitter_favourites INT,twitter_followers INT,twitter_following INT,twitter_lists INT,
twitter_statuses INT,id INT,symbol TEXT,
PRIMARY KEY(id,time,symbol))
WITH CLUSTERING ORDER BY (time ASC);
"""
insert_into_social_status_day_hist = """
(
analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,
code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_stars,code_repo_subscribers,
comments,fb_likes,fb_talking_about,followers,forum_page_views,influence_page_views,markets_page_views,
overview_page_views,points,posts,reddit_active_users,reddit_comments_per_day,reddit_comments_per_hour,
reddit_posts_per_day,reddit_posts_per_hour,reddit_subscribers,time,total_page_views,trades_page_views,
twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses,id,symbol
) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""

# Fetching Vars
url = 'https://min-api.cryptocompare.com/data/social/coin/histo/day?'
sortOrder = 'Latest'
categories = 'ALL_NEWS_CATEGORIES'

In [387]:
# fetch for top10 social status Data 30 days Cryptocurrencies
for sym in top10:
    f3 = ccMaster(url,key_space,api_key,sym)
    social_status_h_Json = f3.getJSON()
    social_status_h_PD = f3.jsonToPandas(social_status_h_Json['Data'])
    prepared10,session10 = f3.pdToCassandra('social_status_day_hist',create_table_social_status_day_hist,insert_into_social_status_day_hist)
    for index, df_iter in social_status_h_PD.iterrows():
            session10.execute(prepared10,(\
                df_iter['analysis_page_views'],df_iter['charts_page_views'],df_iter['code_repo_closed_issues'],\
                df_iter['code_repo_closed_pull_issues'],df_iter['code_repo_forks'],df_iter['code_repo_open_issues'],\
                df_iter['code_repo_open_pull_issues'],df_iter['code_repo_stars'],df_iter['code_repo_subscribers'],\
                df_iter['comments'],df_iter['fb_likes'],df_iter['fb_talking_about'],df_iter['followers'],\
                df_iter['forum_page_views'],df_iter['influence_page_views'],df_iter['markets_page_views'],\
                df_iter['overview_page_views'],df_iter['points'],df_iter['posts'],df_iter['reddit_active_users'],\
                df_iter['reddit_comments_per_day'],df_iter['reddit_comments_per_hour'],\
                df_iter['reddit_posts_per_day'],df_iter['reddit_posts_per_hour'],df_iter['reddit_subscribers'],\
                f3.convertToPst(df_iter['time']),
                df_iter['total_page_views'],df_iter['trades_page_views'],df_iter['twitter_favourites'],\
                df_iter['twitter_followers'],df_iter['twitter_following'],df_iter['twitter_lists'],\
                df_iter['twitter_statuses'],df_iter['id'],df_iter['symbol']))

https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=1182&limit=1513
https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=7605&limit=1513
https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=5031&limit=1513
https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=202330&limit=1513
https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=166503&limit=1513
https://min-api.cryptocompare.com/data/social/coin/histo/day?&api_key=23489088ccc5e95cef763cbedd2d27588a979595edb097f53f40ad7d76239d41&coinId=321992&limit=1513
https://min-api.cryptocompare.com/data/social/

In [361]:
# DB realted variables FOR social_status_day_hist

create_table_all_hist = """(close FLOAT,high FLOAT,low FLOAT,open FLOAT,time TEXT,volumefrom DOUBLE,
volumeto DOUBLE,id FLOAT,symbol TEXT,PRIMARY KEY(id,time,symbol))WITH CLUSTERING ORDER BY (time ASC);"""

In [385]:
# Get open, high, low, close, volumefrom and volumeto from the daily historical data.The values are based on 00:00 GMT time
url = 'https://min-api.cryptocompare.com/data/histoday?'

for sym in top10:
    f4 = ccMaster(url,key_space,fsym=sym,tsym='USD')
    hist_val_Json = f4.getJSON()
    hist_val_PD = f4.jsonToPandas(hist_val_Json['Data'])
    col_names_insert = ','.join(hist_val_PD.columns.tolist())
    qmarks = '?' + ''.join([',?' for i in range(len(hist_val_PD.columns.tolist())-1)])
    col_names_insert = '(' + col_names_insert + ' ) VALUES( ' + qmarks + ')'
    prepared10,session10 = f4.pdToCassandra('all_coins_hist_data',create_table_all_hist,col_names_insert)
    for index, df_iter in hist_val_PD.iterrows():
            session10.execute(prepared10,(\
                df_iter['close'],df_iter['high'],df_iter['low'],
                df_iter['open'],f4.convertToPst(df_iter['time']),
                df_iter['volumefrom'],df_iter['volumeto'],
                df_iter['id'],df_iter['symbol']))

https://min-api.cryptocompare.com/data/histoday?&fsym=BTC&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=ETH&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=XRP&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=BCH&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=EOS&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=ADA&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=LTC&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=XLM&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=IOT&tsym=USD&limit=1513
https://min-api.cryptocompare.com/data/histoday?&fsym=NEO&tsym=USD&limit=1513


In [45]:
# INSERT TOP 10 DATA

keyS=[]
coinID = []
cryptoname = []
imgUrls = []
cluster = Cluster(['127.0.0.1'],load_balancing_policy=None)
session = cluster.connect()
session.set_keyspace(key_space)

insert_query = """INSERT INTO top10data (maintainer,sortorder,symbol,coin_id,coinname,imageurl,firstalpha) VALUES (?,?,?,?,?,?,?)"""
prepared = session.prepare(insert_query)
    
for k in range(len(top10)):
    keyS.append(top10[k])
    query = "SELECT id,coinname,imageurl FROM all_coins WHERE symbol='" + top10[k] + "' ALLOW FILTERING"
    future = session.execute(query)
    coinID.append(future.one()[0])
    cryptoname.append(future.one()[1])
    imgUrls.append(future.one()[2])
    
    
    session.execute(prepared,(1,(k+1),top10[k],future.one()[0],future.one()[1],future.one()[2],future.one()[1][0].lower()))

df = pd.DataFrame({"symbol":keyS,"coin_id":coinID,"coinname":cryptoname,"imageurl":imgUrls})
df['firstalpha'] = df['coinname'].apply(lambda x: x[0].lower())
df


#CREATE TABLE top10data( symbol TEXT, coin_id INT, coinname TEXT, imageurl TEXT, firstalpha TEXT, PRIMARY KEY (symbol,coin_id) );

,symbol,coin_id,coinname,imageurl,firstalpha
0,BTC,1182,Bitcoin,/media/19633/btc.png,b
1,XRP,5031,Ripple,/media/34477776/xrp.png,r
2,ETH,7605,Ethereum,/media/20646/eth_logo.png,e
3,LTC,3808,Litecoin,/media/35309662/ltc.png,l
4,BCH,202330,Bitcoin Cash,/media/1383919/12-bitcoin-cash-square-crop-sma...,b
5,EOS,166503,EOS.IO,/media/1383652/eos_1.png,e
6,ADA,321992,Cardano,/media/12318177/ada.png,c
7,XLM,4614,Stellar,/media/20696/str.png,s
8,IOT,127356,IOTA,/media/1383540/iota_logo.png,i
9,NEO,27368,NEO,/media/1383858/neo.jpg,n


In [7]:
cluster = Cluster(['127.0.0.1'],load_balancing_policy=None)
session = cluster.connect()
session.set_keyspace(key_space) 

In [8]:
# Cassandra to CSV
def cassandraToCsv():
    query = "SELECT * FROM all_coins_hist_data;"
    future = session.execute(query)
    
    return future
    
df = pd.DataFrame(list(cassandraToCsv()))
    
df.to_csv('AllCoinHistData.csv',index=False)

In [149]:
csv_file_name = 'all_coins.csv'
all_coin_csv = pd.read_csv(csv_file_name)
#replace space and .

dfCsv = pd.DataFrame(all_coin_csv).drop(['ProofType'], axis=1)
dfCsv['Algorithm'] = dfCsv['Algorithm'].fillna('NA')
dfCsv['ImageUrl'] = dfCsv['ImageUrl'].fillna('NA')
dfCsv['TotalCoinSupply'] = dfCsv['TotalCoinSupply'].fillna('0')
dfCsv['TotalCoinsMined'] = dfCsv['TotalCoinsMined'].fillna(0.0)
dfCsv['NetHashesPerSecond'] = dfCsv['NetHashesPerSecond'].fillna(0.0)
dfCsv['BlockReward'] = dfCsv['BlockReward'].fillna(0.0)
dfCsv['BlockNumber'] = dfCsv['BlockNumber'].fillna(0)
dfCsv['BlockTime'] = dfCsv['BlockTime'].fillna(0)
dfCsv['Sponsored'][0]

False

In [150]:
cluster = Cluster(['127.0.0.1'],load_balancing_policy=None)
session = cluster.connect()
session.set_keyspace(key_space)

In [151]:
session.execute("""CREATE TABLE IF NOT EXISTS all_coins (
            Id INT,
            Url TEXT,
            ImageUrl TEXT,
            Name TEXT,
            Symbol TEXT,
            CoinName TEXT,
            FullName TEXT,
            Algorithm TEXT,
            FullyPremined INT,
            TotalCoinSupply DOUBLE,
            SortOrder INT,
            Sponsored Boolean,
            IsTrading Boolean,
            TotalCoinsMined DOUBLE,
            BlockNumber FLOAT,
            NetHashesPerSecond DOUBLE,
            BlockReward FLOAT,
            BlockTime FLOAT,
            PRIMARY KEY(Id))
            """)

In [152]:
query = """INSERT INTO all_coins (
        id,url,imageurl,name,symbol,coinname,fullname,algorithm,fullypremined,
        totalcoinsupply,sortorder,sponsored,istrading,totalcoinsmined,blockNumber,
        netHashespersecond,blockreward,blocktime
        ) VALUES
        (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
cnt = 0

prepared = session.prepare(query)
dfCsv['BlockTime'] = dfCsv['BlockTime'].fillna(0.0)
for index, df_iter in dfCsv.iterrows():
    cnt = cnt + 1
#     print(cnt)
    session.execute(prepared,\
                     (df_iter['Id'], df_iter['Url'], df_iter['ImageUrl'], df_iter['Name'], df_iter['Symbol'],\
                     df_iter['CoinName'], df_iter['FullName'], df_iter['Algorithm'], df_iter['FullyPremined'], float(df_iter['TotalCoinSupply']),\
                     df_iter['SortOrder'], df_iter['Sponsored'],df_iter['IsTrading'], df_iter['TotalCoinsMined'], df_iter['BlockNumber'],\
                     df_iter['NetHashesPerSecond'], df_iter['BlockReward'],df_iter['BlockTime']))